In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

# Определить потенциально отточных клиентов

    У нашей компании есть клиенты.
    Они оплачивают наши услуги по различным тарифам. 
    У этих тарифов разная стоимость, разный срок действия и разные условия. Подробнее можно ознакомится здесь - https://www.b2b-center.ru/app/tariffs/?group=supplier.
    Нашей компании важно знать, собирается ли клиент продлевать свой тариф или же перестанет пользоваться нашими услугами ("оттечет"). 
    Большинство тарифов квартальные и поэтому далее будем говорить об оплатах в рамках квартала.
    
    
    

# Тестовое задание
    Тестовое задание моделирует (упрощенно) реальную задачу прогноза оплаты/оттока клиента и формат задачи выполнен аналогично задаче на kaggle.
    Допустим мы находимся в 1 декабря 2022 года (все тренировочные данные заканчиваются этой датой)
    
    Нам важно знать кто из клиентов заплатит нам в 1 квартале 2023 года. Точное значение получить невозможно, поэтому хочется оценить "вероятность" этой оплаты.
    
    
    В качестве проверочной метрики будет использоваться roc_auc_score, но интересно будет обсудить и альтернативные подходы к выбору метрики.

# Данные

In [2]:

""" 
В файле train_action_df.csv содержится информация о важнейших действиях на нашем сайте за последние 2 года. 

поля:
    user_id - уникальный идентификатор клиента
    action_type - 5 типов важнейших действий на нашем сайте. Категориальная переменная (action_4 НЕ обязательно важнее, чем action_3 и т.д.)
    action_date - дата совершения действия
    cnt_actions - количество действий данного типа, совершенных в эту дату

"""

train_action_df = pd.read_csv('train_action_df.csv')
train_action_df.head()

,user_id,action_type,action_date,cnt_actions
0,user_26091,action_2,2020-12-04,1
1,user_26091,action_2,2020-12-17,1
2,user_26091,action_2,2020-12-21,1
3,user_26091,action_2,2021-01-13,1
4,user_26091,action_2,2021-01-15,1


In [3]:

""" 
В файле train_pay_df содержится информация об оплатах за последние 2 года. 

поля:
    user_id - уникальный идентификатор клиента
    pay_date - дата совершения оплаты (НЕ всегда соответствует оплаченному периоду!!!)
    year - год действия тарифа
    quarter - квартал действия тарифа
    tariff - обезличенный тип тарифа 
    

Небольшие пояснения.
Довольно часто клиенты оплачивают тариф заранее. Например, 1 ноября 2022 года могут оплатить 1 квартал 2023 года.
    
"""

train_pay_df = pd.read_csv('train_pay_df.csv')
train_pay_df.head()




,user_id,pay_date,year,quarter,period,tariff
0,user_50838,2021-01-03,2021,1,205,tariff_15
1,user_56500,2021-01-05,2021,1,205,tariff_20
2,user_50795,2021-01-05,2021,1,205,tariff_25
3,user_4718,2021-01-10,2021,1,205,tariff_10
4,user_22255,2021-01-10,2021,1,205,tariff_10


# Результат

In [4]:
# в файле test_df находятся клиенты, для которых необходимо сделать прогноз
test_df = pd.read_csv('test_df.csv')

# результатом модель должна выдать вероятность (proba) для каждой строчки из этого файла
# я заполню случайно для понимания формата файла
test_df['proba'] = np.random.uniform(size = test_df.shape[0])

In [5]:
test_df.head()

,user_id,year,quarter,period,proba
0,user_44331,2022,4,212,0.882455
1,user_49203,2022,4,212,0.962668
2,user_21597,2022,4,212,0.442041
3,user_36314,2022,4,212,0.576818
4,user_45092,2022,4,212,0.085728


In [6]:
# в качестве результата достаточно прислать файл с 2-мя столбцами
test_df[['user_id','proba']].to_csv('predict_df.csv', index = False)

# Связь test_df и обучающих данных

In [7]:
# test_df может быть получен из train_pay_df следующим скриптом:
copy_test_df = train_pay_df[(train_pay_df['year']==2022) & 
             (train_pay_df['quarter']==4)][['user_id','year','quarter','period']].drop_duplicates().reset_index(drop=True)

assert copy_test_df.shape[0]==copy_test_df.merge(test_df).shape[0]

In [8]:
copy_test_df.head()

,user_id,year,quarter,period
0,user_44331,2022,4,212
1,user_49203,2022,4,212
2,user_21597,2022,4,212
3,user_36314,2022,4,212
4,user_45092,2022,4,212


# Проверка

In [9]:
# при проверке я возьму реальные данные об оплатах каждым пользователем 
# хотя бы 1 раз period = 213 (year = 2023, quarter = 1)
# Если пользователь заплатил, то target = 1, если нет = 0.

# для примера сейчас тоже заполню случайно
test_df['target'] = np.random.randint(0, 2, size = test_df.shape[0])

print('Финальная метрика', roc_auc_score(test_df['target'], test_df['proba']))

Финальная метрика 0.4962176896999742


# Решение

## Кроме файла с прогнозом (predict_df.csv) крайне желательно прислать ноутбук или скрипт, которым это решение было получено для понимания уровня кандидата